In [5]:
#  fully workin g code 
from google.cloud import bigquery

client = bigquery.Client()

sql_query = """
    SELECT * FROM `audio-quality-analysis.audio_quality_scoring.AudioSegmentScoringDetails` WHERE __metadata__.run_id = '62568e28-e1e1-11ee-9b36-0e09fceeedd1'
"""
query_job = client.query(sql_query)
df = query_job.to_dataframe()


#  calculate quality issue 
quality_issues_count = (
    df.groupby(['recording_id', 'primary_quality_issue'])
    .size()
    .reset_index(name='issue_count')
    .assign(rank=lambda x: x.groupby('recording_id')['issue_count'].rank(method='dense', ascending=False))
)
excel_file_path = "quality_issues_count.xlsx"
quality_issues_count.to_excel(excel_file_path, index=False)

#  calculate segment score :
segment_score = df.assign(
    prim_score=lambda x: round((x['primary_quality_confidence'] / 100) * 5, 1),
    sec_score=lambda x: round((x['secondary_quality_confidence'] / 100) * 5, 1),
    tert_score=lambda x: round((x['tertiary_quality_confidence'] / 100) * 5, 1)
)[['recording_id', 'recording_duration', 'segment_number',
   'primary_quality_issue', 'secondary_quality_issue', 'tertiary_quality_issue',
   'prim_score', 'sec_score', 'tert_score']]

# Sort the DataFrame
segment_score = segment_score.sort_values(by=['recording_id', 'segment_number'])
excel_file_path = "segment_score.xlsx"
segment_score.to_excel(excel_file_path, index=False)

# calculate seg score count 
seg_scores_count = segment_score.assign(
    issues_count=lambda x: (x['prim_score'] > 0.0).astype(int) +
                           (x['sec_score'] > 0.0).astype(int) +
                           (x['tert_score'] > 0.0).astype(int)
)[['recording_id', 'recording_duration', 'segment_number',
   'primary_quality_issue', 'secondary_quality_issue', 'tertiary_quality_issue',
   'prim_score', 'sec_score', 'tert_score', 'issues_count']]
excel_file_path = "segment_score_count.xlsx"
seg_scores_count.to_excel(excel_file_path, index=False)
print(seg_scores_count)


def calculate_avg_score(row):
    if row['primary_quality_issue'] != 'Normal':
        return round((5 - 0.4 * (row['prim_score'] + row['sec_score'] + row['tert_score']) / row['issues_count']), 2)
    else:
        return round(row['prim_score'], 2)
    
    
    

# Apply the function to create the avg_score column
seg_scores_count['avg_score'] = seg_scores_count.apply(calculate_avg_score, axis=1)
seg_avg_score = seg_scores_count[['recording_id', 'recording_duration', 'segment_number', 'primary_quality_issue',
                              'secondary_quality_issue', 'tertiary_quality_issue', 'prim_score', 'sec_score',
                              'tert_score', 'avg_score']]
excel_file_path = "seg_avg_score.xlsx"
seg_avg_score.to_excel(excel_file_path, index=False)



# Group by recording_id and recording_duration, and calculate the aggregate audio quality score
agg_audio_quality_score = seg_avg_score.groupby(['recording_id', 'recording_duration']).apply(
    lambda x: round(x['avg_score'].sum() / x['segment_number'].max(), 2)).reset_index(name='audio_quality_score')

excel_file_path = "agg_audio_quality_score.xlsx"
agg_audio_quality_score.to_excel(excel_file_path, index=False)





#  final rs
import pandas as pd

# Assuming you have DataFrames named agg_audio_quality_score and quality_issues_count

# Join the two DataFrames on recording_id
merged_df = pd.merge(agg_audio_quality_score, quality_issues_count, on='recording_id', how='inner')

# Define a function to retrieve the primary issue based on rank
def get_primary_issue(rank_series):
    return rank_series[rank_series['rank'] == 1]['primary_quality_issue'].max()

# Group by recording_id, recording_duration, and audio_quality_score
grouped_df = merged_df.groupby(['recording_id', 'recording_duration', 'audio_quality_score']).apply(get_primary_issue).reset_index()

# Rename the columns
grouped_df.columns = ['recording_id', 'recording_duration', 'audio_quality_score', 'primary_audio_issue']

# Get secondary and tertiary audio issues
secondary_audio_issues = merged_df[merged_df['rank'] == 2].groupby(['recording_id', 'recording_duration', 'audio_quality_score'])['primary_quality_issue'].max().reset_index()
tertiary_audio_issues = merged_df[merged_df['rank'] == 3].groupby(['recording_id', 'recording_duration', 'audio_quality_score'])['primary_quality_issue'].max().reset_index()

# Merge secondary and tertiary audio issues into grouped_df
grouped_df = pd.merge(grouped_df, secondary_audio_issues, on=['recording_id', 'recording_duration', 'audio_quality_score'], how='left')
grouped_df = pd.merge(grouped_df, tertiary_audio_issues, on=['recording_id', 'recording_duration', 'audio_quality_score'], how='left')

# Rename the columns
grouped_df.rename(columns={'primary_quality_issue_x': 'secondary_audio_issue',
                           'primary_quality_issue_y': 'tertiary_audio_issue'}, inplace=True)

# Add the created timestamp column
grouped_df['created_ts'] = pd.Timestamp.now()

# Sort the DataFrame by recording_id
grouped_df.sort_values(by='recording_id', inplace=True)

# Print the resulting DataFrame
# print(grouped_df)
grouped_df.to_excel('audio_quality_results.xlsx', index=False)

print("Result saved to audio_quality_results.xlsx")



















                                  recording_id  recording_duration  \
4110  045a01f9-d846-11ee-8055-ac198ecfef08.wav              67.000   
3754  045a01f9-d846-11ee-8055-ac198ecfef08.wav              67.000   
3786  045a01f9-d846-11ee-8055-ac198ecfef08.wav              67.000   
3732  045a01fa-d846-11ee-9923-ac198ecfef08.wav             118.000   
3747  045a01fa-d846-11ee-9923-ac198ecfef08.wav             118.000   
...                                        ...                 ...   
4900                             normal_84.wav             198.879   
986                              normal_84.wav             198.879   
3616                             normal_84.wav             198.879   
1462                              normal_9.wav              39.000   
372                               normal_9.wav              39.000   

      segment_number primary_quality_issue secondary_quality_issue  \
4110               1                  echo              low_volume   
3754               

In [ ]:
!gcloud auth login

In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:\\Users\\YashashreeAdhangle\\AppData\\Roaming\\gcloud\\application_default_credentials.json"

In [ ]:
!gcloud auth application-default login

AttributeError: 'str' object has no attribute 'to_excel'

In [9]:
#  working but neatly arrange 
from google.cloud import bigquery
import pandas as pd

# Initialize BigQuery client
client = bigquery.Client()

# Define the SQL query to retrieve data from BigQuery
sql_query = """
    SELECT * 
    FROM `audio-quality-analysis.audio_quality_scoring.AudioSegmentScoringDetails` 
    WHERE __metadata__.run_id = '62568e28-e1e1-11ee-9b36-0e09fceeedd1'
"""

# Execute the SQL query and convert the result to a DataFrame
query_job = client.query(sql_query)
df = query_job.to_dataframe()

# Calculate quality issues count
quality_issues_count = (
    df.groupby(['recording_id', 'primary_quality_issue'])
    .size()
    .reset_index(name='issue_count')
    .assign(rank=lambda x: x.groupby('recording_id')['issue_count'].rank(method='dense', ascending=False))
)


# quality_issues_count.to_excel("quality_issues_count.xlsx", index=False)

# Calculate segment score
segment_score = df.assign(
    prim_score=lambda x: round((x['primary_quality_confidence'] / 100) * 5, 1),
    sec_score=lambda x: round((x['secondary_quality_confidence'] / 100) * 5, 1),
    tert_score=lambda x: round((x['tertiary_quality_confidence'] / 100) * 5, 1)
)[['recording_id', 'recording_duration', 'segment_number',
   'primary_quality_issue', 'secondary_quality_issue', 'tertiary_quality_issue',
   'prim_score', 'sec_score', 'tert_score']]

# Sort the segment score DataFrame
segment_score = segment_score.sort_values(by=['recording_id', 'segment_number'])

# Save segment score to an Excel file
# segment_score.to_excel("segment_score.xlsx", index=False)

# Calculate segment score count
seg_scores_count = segment_score.assign(
    issues_count=lambda x: (x['prim_score'] > 0.0).astype(int) +
                           (x['sec_score'] > 0.0).astype(int) +
                           (x['tert_score'] > 0.0).astype(int)
)[['recording_id', 'recording_duration', 'segment_number',
   'primary_quality_issue', 'secondary_quality_issue', 'tertiary_quality_issue',
   'prim_score', 'sec_score', 'tert_score', 'issues_count']]

# Save segment score count to an Excel file
# seg_scores_count.to_excel("segment_score_count.xlsx", index=False)

# Calculate average score for segments
def calculate_avg_score(row):
    if row['primary_quality_issue'] != 'Normal':
        return round((5 - 0.4 * (row['prim_score'] + row['sec_score'] + row['tert_score']) / row['issues_count']), 2)
    else:
        return round(row['prim_score'], 2)

seg_scores_count['avg_score'] = seg_scores_count.apply(calculate_avg_score, axis=1)
seg_avg_score = seg_scores_count[['recording_id', 'recording_duration', 'segment_number', 'primary_quality_issue',
                              'secondary_quality_issue', 'tertiary_quality_issue', 'prim_score', 'sec_score',
                              'tert_score', 'avg_score']]

# Save segment average score to an Excel file
# seg_avg_score.to_excel("seg_avg_score.xlsx", index=False)

# Group by recording_id and recording_duration, and calculate the aggregate audio quality score
agg_audio_quality_score = seg_avg_score.groupby(['recording_id', 'recording_duration']).apply(
    lambda x: round(x['avg_score'].sum() / x['segment_number'].max(), 2)).reset_index(name='audio_quality_score')

# Save aggregate audio quality score to an Excel file
# agg_audio_quality_score.to_excel("agg_audio_quality_score.xlsx", index=False)

# Merge quality issues count and aggregate audio quality score DataFrames
merged_df = pd.merge(agg_audio_quality_score, quality_issues_count, on='recording_id', how='inner')

# Define a function to retrieve the primary issue based on rank
def get_primary_issue(rank_series):
    return rank_series[rank_series['rank'] == 1]['primary_quality_issue'].max()

# Group by recording_id, recording_duration, and audio_quality_score
grouped_df = merged_df.groupby(['recording_id', 'recording_duration', 'audio_quality_score']).apply(get_primary_issue).reset_index()

# Merge secondary and tertiary audio issues into grouped_df
secondary_audio_issues = merged_df[merged_df['rank'] == 2].groupby(['recording_id', 'recording_duration', 'audio_quality_score'])['primary_quality_issue'].max().reset_index()
tertiary_audio_issues = merged_df[merged_df['rank'] == 3].groupby(['recording_id', 'recording_duration', 'audio_quality_score'])['primary_quality_issue'].max().reset_index()
grouped_df = pd.merge(grouped_df, secondary_audio_issues, on=['recording_id', 'recording_duration', 'audio_quality_score'], how='left')
grouped_df = pd.merge(grouped_df, tertiary_audio_issues, on=['recording_id', 'recording_duration', 'audio_quality_score'], how='left')

# Rename the columns
grouped_df.rename(columns={'primary_quality_issue_x': 'secondary_audio_issue',
                           'primary_quality_issue_y': 'tertiary_audio_issue'}, inplace=True)

# Add the created timestamp column
grouped_df['created_ts'] = pd.Timestamp.now()

# Sort the DataFrame by recording_id
grouped_df.sort_values(by='recording_id', inplace=True)

# Save the resulting DataFrame to an Excel file
grouped_df.to_excel("audio_quality_results.xlsx", index=False)

print("Result saved to audio_quality_results.xlsx")


Result saved to audio_quality_results.xlsx
